## Lesley wants to know how many IGSNs there are

In [ ]:
import json
import os
import requests

use_test_api = False

with open('datacite-api-config.json') as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]

if use_test_api:
    api_endpoint = 'https://api.test.datacite.org/'
else:
    api_endpoint = 'https://api.datacite.org/'

In [ ]:
clientTypes = ['igsnCatalog']

total = 0

path = 'reports'
if not os.path.exists(path):
    os.mkdir(path)
    
url = api_endpoint
headers = {
    'Content-Type': 'application/vnd.api+json',
}

for clientType in clientTypes:
    url = api_endpoint + 'clients?page[number]=1&page[size]=1000&client-type=' + clientType
    print('Getting information for ' + clientType)
    response = requests.request("GET", url, auth=(username, password), headers = headers)
    clients = json.loads(response.text)
    print('Writing to ' + clientType + '/clients.json')
    if not os.path.exists(path + '/'+ clientType):
        os.mkdir(path + '/'+ clientType)
    with open(path + '/'+ clientType + '/clients.json', 'w+') as jsonfile:
        json.dump(clients, jsonfile, indent=3)
    print(str(len(data['data'])) + ' clients found')
    for client in clients['data']:
        clientName = client['attributes']['name']
        clientID = client['attributes']['symbol']
        url = api_endpoint + 'dois?page[number]=1&page[size]=1&client-id=' + clientID
        response = requests.request("GET", url, auth=(username, password), headers = headers)
        clientData = json.loads(response.text)
        print(clientName + ' (' + client['attributes']['symbol'] + '):\t' + str(clientData['meta']['total']))
        total += clientData['meta']['total']
        if not os.path.exists(path + '/'+ clientType):
            os.mkdir(path + '/'+ clientType)
        with open(path + '/'+ clientType + '/' + clientID + '.json', 'w+') as jsonfile:
            json.dump(clientData, jsonfile, indent=3)
print('Found ' + str(total) + ' DOIs')
print('Done.')


In [ ]:
print('Found ' + str(total) + ' DOIs')